<a href="https://colab.research.google.com/github/softwareuhuy/take-api/blob/main/Tutorial_Validasi_Data_Python_by_LUNCHTIMEMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

# baca path gdrive
# SHEET YANG DIDOWNLOAD BERFORMAT 00:00 GAK USAH GMT+7
df=pd.read_csv('/content/gdrive/My Drive/Reyhan/Deli/25_FEB-09_OKT.csv')

df = df.loc[:,['created_at','entry_id','field1','field2','field3','field4','field5','field6','field7']]

# HILANGIN TULISAN T DAN Z YANG ADA
df['created_at']=df['created_at'].str.replace("T"," ")
df['created_at']=df['created_at'].str.replace("Z"," ")

df.rename(columns={'field1': 'Temperatur', 'field2': 'Kelembapan Relatif','field3': 'PM2.5', 'field4' : 'CO2', 'field5' : 'Tekanan Udara','field6' : 'Temperatur SHT31','field7' : 'Humidity SHT31'}, inplace=True)
df = df.dropna()
# UBAH time yang ada jadi gmt +7 local time
df['created_at'] = pd.to_datetime(df['created_at']).dt.tz_convert('Asia/Jakarta')

# hilangkan nilai co2 dan pm2.5 yang bernilai <= 0
df=df[(df['CO2']>0)]
df=df[(df['PM2.5']>0)]


df

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,created_at,entry_id,Temperatur,Kelembapan Relatif,PM2.5,CO2,Tekanan Udara,Temperatur SHT31,Humidity SHT31
8485,2023-03-02 14:44:29+07:00,8486,0.0,0.0,2350,93764.20313,0.00000,5.0,0.00
8486,2023-03-02 14:46:31+07:00,8487,0.0,0.0,2350,93764.20313,0.00000,5.0,0.00
8487,2023-03-02 14:48:32+07:00,8488,0.0,23.0,2000,93769.11719,0.00000,3.0,0.00
9280,2023-03-04 13:11:10+07:00,9281,25.2,78.1,11,993.75000,93846.89063,0.0,4.19
9281,2023-03-04 13:13:10+07:00,9282,25.2,78.1,11,993.75000,93846.89063,0.0,4.19
...,...,...,...,...,...,...,...,...,...
156505,2023-10-09 12:12:48+07:00,156506,37.1,26.4,36,840.63000,93731.32813,0.0,4.10
156506,2023-10-09 12:14:48+07:00,156507,37.0,26.5,36,918.75000,93725.50000,0.0,11.89
156507,2023-10-09 12:16:48+07:00,156508,37.0,26.5,36,918.75000,93725.50000,0.0,11.89
156508,2023-10-09 12:18:48+07:00,156509,37.2,26.4,47,518.75000,93712.75781,0.0,6.59


In [ ]:
# Ambil Kelembapan Relatif & PM 2.5 (CEK BULAN,HARI & JAM)
df1 = df.loc[:,['created_at','entry_id','Temperatur','Kelembapan Relatif','PM2.5','CO2']]
df1['Bulan'] = pd.DatetimeIndex(df['created_at']).month
df1.set_index = df1['Bulan'].inplace=True
df1['Hari'] = pd.DatetimeIndex(df['created_at']).day
df1.set_index = df1['Hari'].inplace=True
df1['Jam'] = pd.DatetimeIndex(df['created_at']).hour
df1.set_index = df1['Jam'].inplace=True

df1

,created_at,entry_id,Temperatur,Kelembapan Relatif,PM2.5,CO2,Bulan,Hari,Jam
8476,2023-03-02 14:13:15+07:00,8477,0.0,0.0,0,0.00,3,2,14
8477,2023-03-02 14:16:27+07:00,8478,0.0,0.0,0,0.00,3,2,14
8478,2023-03-02 14:18:30+07:00,8479,0.0,0.0,0,0.00,3,2,14
8480,2023-03-02 14:31:11+07:00,8481,0.0,0.0,0,0.00,3,2,14
8481,2023-03-02 14:33:12+07:00,8482,0.0,0.0,0,0.00,3,2,14
...,...,...,...,...,...,...,...,...,...
156505,2023-10-09 12:12:48+07:00,156506,37.1,26.4,36,840.63,10,9,12
156506,2023-10-09 12:14:48+07:00,156507,37.0,26.5,36,918.75,10,9,12
156507,2023-10-09 12:16:48+07:00,156508,37.0,26.5,36,918.75,10,9,12
156508,2023-10-09 12:18:48+07:00,156509,37.2,26.4,47,518.75,10,9,12


In [ ]:
#Validation
#Hapus nilai CO2 yang di luar batas range
df2 = df1.groupby(pd.Grouper(key='created_at',freq='2min')).mean().reset_index()
df2 = df2[(df2['CO2']>400) &  (df2['CO2'] <5000)]
df2 = df2.loc[:,['created_at','Temperatur','Kelembapan Relatif','CO2','Bulan','Hari']]
df2 = df2.dropna()
df2['CO2']=df2['CO2'].round().astype(int)
df2 = df2.round(2)
df2

,created_at,Temperatur,Kelembapan Relatif,CO2,Bulan,Hari
1407,2023-03-04 13:06:00+07:00,25.0,78.4,750,3.0,4.0
1408,2023-03-04 13:08:00+07:00,25.0,78.4,750,3.0,4.0
1409,2023-03-04 13:10:00+07:00,25.2,78.1,994,3.0,4.0
1410,2023-03-04 13:12:00+07:00,25.2,78.1,994,3.0,4.0
1411,2023-03-04 13:14:00+07:00,25.4,76.6,947,3.0,4.0
...,...,...,...,...,...,...
159060,2023-10-09 12:12:00+07:00,37.1,26.4,841,10.0,9.0
159061,2023-10-09 12:14:00+07:00,37.0,26.5,919,10.0,9.0
159062,2023-10-09 12:16:00+07:00,37.0,26.5,919,10.0,9.0
159063,2023-10-09 12:18:00+07:00,37.2,26.4,519,10.0,9.0


In [ ]:
#MONTHS CO2
path = '/content/gdrive/My Drive/Reyhan/Training/'

x=1
while x<=12 :
  bul = str(x)
  #har = str(y)
  df3 = df2[(df2['Bulan']==x)]
  df4 = df3.loc[:,['created_at','Kelembapan Relatif','CO2']]


  data = len(df3.axes[0])
  jumlah = str(data)
  if (df3.empty==True):
    pass
  else :
    Q3 = df4['CO2'].describe()['75%']
    Q1 = df4['CO2'].describe()['25%']
    IQR = Q3 - Q1
    UF = Q3 + (1.5*IQR)
    LF = Q1 - (1.5*IQR)
    df4 = df4[(df2['CO2'] < UF) & (df2['CO2'] > LF)]
    df4 = df4.groupby(pd.Grouper(key='created_at',freq='24H')).mean().reset_index()
    df5 = df4.loc[:,['CO2']]
    std = df5['CO2'].std(numeric_only=True).round(2)
    mean = df5['CO2'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    df4 = df4.round(2)
    df4.to_csv(path+"Bulan "+bul+" | "+"SD: "+sd+" ppm "+ "CV: " +koef+"% | "+"CO2 Validation.csv", index=None)
  x+=1

<ipython-input-5-a5b79dd84aa8>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df4 = df4[(df2['CO2'] < UF) & (df2['CO2'] > LF)]
<ipython-input-5-a5b79dd84aa8>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df4 = df4[(df2['CO2'] < UF) & (df2['CO2'] > LF)]
<ipython-input-5-a5b79dd84aa8>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df4 = df4[(df2['CO2'] < UF) & (df2['CO2'] > LF)]
<ipython-input-5-a5b79dd84aa8>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df4 = df4[(df2['CO2'] < UF) & (df2['CO2'] > LF)]
<ipython-input-5-a5b79dd84aa8>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df4 = df4[(df2['CO2'] < UF) & (df2['CO2'] > LF)]
<ipython-input-5-a5b79dd84aa8>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df4 = df4[(df2['CO2'] < UF) & (df2['CO2'] > LF)]
<ipython-input-5-a5b79dd84aa

In [ ]:
#24H co2
path = '/content/gdrive/My Drive/Reyhan/Deli/Hari/'
x=1


while x<=12 :
  y=1
  while y<=31:
    bul = str(x)
    har = str(y)
    #har = str(y
    df3 = df2[(df2['Bulan']==x) & (df2['Hari']==y) ]
    df4 = df3.loc[:,['created_at','Kelembapan Relatif','CO2']]

    data = len(df3.axes[0])
    jumlah = str(data)
    if (df3.empty==True or data < 540):
      pass
    else :
      Q3 = df4['CO2'].describe()['75%']
      Q1 = df4['CO2'].describe()['25%']
      IQR = Q3 - Q1
      UF = Q3 + (1.5*IQR)
      LF = Q1 - (1.5*IQR)
      df4 = df4[(df2['CO2'] < UF) & (df2['CO2'] > LF)]
      df4 = df4.groupby(pd.Grouper(key='created_at',freq='1H')).mean().reset_index()
      df5 = df4.loc[:,['CO2']]
      std = df5['CO2'].std(numeric_only=True).round(2)
      mean = df5['CO2'].mean()
      cv = ((std / mean)*100).round(2)
      sd = str(std)
      koef = str(cv)
      df4 = df4.round(2)
      df4.to_csv(path+"Bulan "+bul+" Hari "+har+" | "+"SD: "+sd+" ppm "+ "CV: " +koef+"% | "+"CO2 Validation.csv", index=None)
    y+=1
  x+=1

In [ ]:
#24 hour average CO2
counts = df1['entry_id'].value_counts()
print(counts)

df5=df1[~df1['entry_id'].isin(counts[counts<540].index)].round(2)
if df5.empty :
  print("No Valid Data")

else :
  df5['Date_time']=pd.to_datetime(df.created_at)
  df5 = df5.groupby(pd.Grouper(key='created_at',freq='24H')).mean()
  df5 = df5.dropna()
  df5.reset_index(inplace=True)

  df5=df5[df5['CO2'].notna()]
  df5['CO2']=df5['CO2'].round().astype(int)
  df5 = df5.fillna('-')
  df5

In [ ]:
##Validation PM2.5
path = '/content/gdrive/My Drive/Reyhan/Deli/Bulan/'
#Remove negative values and max range test for PM2.5
df2 = df1.groupby(pd.Grouper(key='created_at',freq='2min')).mean().reset_index()

df2 = df2[(df2['PM2.5']> 0) &  (df2['PM2.5'] < 250)]
#Correction factor for RH >= 80
df2.loc[df2['Kelembapan Relatif']>=80, ['PM2.5']] *= 0.67
df2 = df2.loc[:,['created_at','Kelembapan Relatif','PM2.5','Bulan','Hari']]
df2 = df2.dropna()
#Remove PM2.5 value if the Vcc is higher than 6 V for PM2.5
#df3 = df3[(df3['V PM2.5'] < 6)]
df2['PM2.5']=df2['PM2.5'].round().astype(int)
df2.round(2)
df2.to_csv(path+"PM2.5 dummy.csv", index=None)


In [ ]:
#VALID months PM2.5
path = '/content/gdrive/My Drive/Reyhan/Deli/Bulan/'

x=1
while x<=12 :
  bul = str(x)
  df3 = df2[(df2['Bulan']==x)]
  df4 = df3.loc[:,['created_at','Kelembapan Relatif','PM2.5']]


  jumlah = str(data)

  Q3 = df4['PM2.5'].describe()['75%']
  Q1 = df4['PM2.5'].describe()['25%']
  IQR = Q3 - Q1
  UF = Q3 + (1.5*IQR)
  LF = Q1 - (1.5*IQR)
  df4 = df4[(df2['PM2.5'] < UF) & (df2['PM2.5'] > LF)]
  df4 = df4.groupby(pd.Grouper(key='created_at',freq='24H')).mean().reset_index()


  if (df4.empty==True):
    pass
  else :
    df5 = df4.loc[:,['PM2.5']]
    std = df5['PM2.5'].std(numeric_only=True).round(2)
    mean = df5['PM2.5'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    df4.round(2)
    df4.to_csv(path+"Bulan "+bul+" | "+"SD: "+sd+" ug_m^3 "+ "CV: " +koef+"% | "+"PM2.5 Validation.csv", index=None)
  x+=1

In [ ]:
#24H PM2.5
path = '/content/gdrive/My Drive/Reyhan/Deli/Hari/'
x=1


while x<=12 :
  y=1
  while y<=31:
    bul = str(x)
    har = str(y)
    #har = str(y
    df3 = df2[(df2['Bulan']==x) & (df2['Hari']==y)]
    df4 = df3.loc[:,['created_at','Kelembapan Relatif','PM2.5']]

    data = len(df4.axes[0])
    jumlah = str(data)
    if (df4.empty==True or data <540):
      pass
    else :
      Q3 = df4['PM2.5'].describe()['75%']
      Q1 = df4['PM2.5'].describe()['25%']
      IQR = Q3 - Q1
      UF = Q3 + (1.5*IQR)
      LF = Q1 - (1.5*IQR)
      df4 = df4[(df2['PM2.5'] < UF) & (df2['PM2.5'] > LF)]
      df4 = df4.groupby(pd.Grouper(key='created_at',freq='1H')).mean().reset_index()
      df5 = df4.loc[:,['PM2.5']]
      std = df5['PM2.5'].std().round(2)
      mean = df5['PM2.5'].mean()
      cv = ((std / mean)*100).round(2)
      sd = str(std)
      koef = str(cv)
      print (sd)
      print (koef)
      df4.round(2)
      df4.to_csv(path+"Bulan "+bul+" Hari "+har+" | "+"SD: "+sd+" ppm "+ "CV: " +koef+"% | "+"PM2.5 Validation.csv", index=None)
    y+=1
  x+=1

In [ ]:
#windowing outlier detection for CO2 24Hour

def sleding(df, column_name, window_size):
    df1=[]
    if len(elements) >= window_size:
        for i in range(len(elements) - window_size + 1):
            dfe=elements[i:i+window_size]

                  #Outlier Detection
            dfe['CO2'].describe()
            Q3 = dfe['CO2'].describe()['75%']
            Q1 = dfe['CO2'].describe()['25%']
            IQR = Q3 - Q1
            UF = Q3 + (1.5*IQR)
            LF = Q1 - (1.5*IQR)
            dfe= dfe[(dfe['CO2'] < UF) & (dfe['CO2'] > LF)]
            print(UF)
            print(LF)
            if i==0:
                df1=dfe

            else:
                df1=pd.concat(dfe.iloc[-1])
    return df1

In [ ]:
#Outlier Detection
Q3 = df1['CO2'].describe()['75%']
Q1 = df1['CO2'].describe()['25%']
IQR = Q3 - Q1
UF = Q3 + (1.5*IQR)
LF = Q1 - (1.5*IQR)
df5 = df1[(df1['CO2'] < UF) & (df1['CO2'] > LF)]
print(UF)
print(LF)



1160.0
456.0


In [ ]:
#24 hour average CO2
df1['entry_id'] = df1['created_at'].dt.day

counts = df1['entry_id'].value_counts()
print(counts)

df5=df1[~df1['entry_id'].isin(counts[counts<540].index)].round(2)
if df5.empty :
  print("No Valid Data")

else :
  df5['Date_time']=pd.to_datetime(df.created_at)
  df5 = df5.groupby(pd.Grouper(key='created_at',freq='24H')).mean()
  df5 = df5.dropna()
  df5.reset_index(inplace=True)

  df5=df5[df5['CO2'].notna()]
  df5['CO2']=df5['CO2'].round().astype(int)
  df5 = df5.fillna('-')
  df5

1     1070
13     720
29     720
28     720
27     720
20     720
19     720
18     720
17     720
16     720
2      720
14     720
15     720
12     720
9      720
8      720
7      720
6      720
5      720
3      720
30     720
4      719
26     707
25     693
23     671
10     480
24     477
21     450
11     233
Name: entry_id, dtype: int64


<ipython-input-11-acf89c185f33>:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df5 = df5.groupby(pd.Grouper(key='created_at',freq='24H')).mean()


# New Section